### CoinMarketCap Study
- API documentation is enable to be read in https://coinmarketcap.com/api/documentation/v1/#

### Health-Check

In [1]:
import requests as r
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os
import pandas as pd

- Testing the API connection in the sandbox environment

In [2]:
test_url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
}

session = r.Session()
session.headers.update(headers)

try:
  response = session.get(test_url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2025-01-02T14:23:45.001Z', 'error_code': 0, 'error_message': None, 'elapsed': 1, 'credit_count': 1, 'notice': None}, 'data': [{'id': 1862, 'name': 'bgmejx6ys5', 'symbol': 'vhi4ef2cj7e', 'slug': 'f76nt9f1drt', 'cmc_rank': 5286, 'num_market_pairs': 145, 'circulating_supply': 1889, 'total_supply': 561, 'max_supply': 5160, 'infinite_supply': None, 'last_updated': '2025-01-02T14:23:45.001Z', 'date_added': '2025-01-02T14:23:45.001Z', 'tags': ['92twp2cucq', 'rcjrcjm63ac', '3jcf2cuvxlg', 'gxj6wdkwf8b', '0s3td18rnaia', 'fo2u579qcnh', 'hmvafacuz97', '1teatf1l78r', 'lx1p2qstexe', 'l4svmy6zua'], 'platform': None, 'self_reported_circulating_supply': None, 'self_reported_market_cap': None, 'quote': {'USD': {'price': 0.33594552966837066, 'volume_24h': 8192, 'volume_change_24h': 0.6893784067602771, 'percent_change_1h': 0.28900017226681607, 'percent_change_24h': 0.4486020260212198, 'percent_change_7d': 0.07207359809951153, 'market_cap': 0.638076277442853, 'market_cap_dominance

- Testing the API connection in production environment with my own user credentials

In [3]:
# defining the base url that will be useful on any later calls
base_url= os.environ.get('COIN_MARKET_API_BASE_URL')

# defining the access_token regarding my own user in the subscribed platform
access_token= os.environ.get('COIN_MARKET_API_ACCESS_TOKEN')

# api call parameters (here using the same as above)
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}

# redefining the headers variable according to documentation reference, but with the appropriate pointer to my user access token
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': access_token,
}

# doing the api call attempt
test_url= base_url+'/v1/cryptocurrency/listings/latest'
try:
    response = r.get(test_url, params= parameters, headers= headers)
    data= json.loads(response.text)
    #print(data) 
    print(response.status_code)
    print(response.reason)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)


200
OK


In [4]:
data

{'status': {'timestamp': '2025-01-02T14:23:46.431Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 81,
  'credit_count': 25,
  'notice': None,
  'total_count': 10492},
 'data': [{'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'slug': 'bitcoin',
   'num_market_pairs': 11853,
   'date_added': '2010-07-13T00:00:00.000Z',
   'tags': ['mineable',
    'pow',
    'sha-256',
    'store-of-value',
    'state-channel',
    'coinbase-ventures-portfolio',
    'three-arrows-capital-portfolio',
    'polychain-capital-portfolio',
    'binance-labs-portfolio',
    'blockchain-capital-portfolio',
    'boostvc-portfolio',
    'cms-holdings-portfolio',
    'dcg-portfolio',
    'dragonfly-capital-portfolio',
    'electric-capital-portfolio',
    'fabric-ventures-portfolio',
    'framework-ventures-portfolio',
    'galaxy-digital-portfolio',
    'huobi-capital-portfolio',
    'alameda-research-portfolio',
    'a16z-portfolio',
    '1confirmation-portfolio',
    'winklevoss-capital-portfol

### Exploring API endpoints and looking for useful content

#### 1) Cryptocurrency metadata
- According to documentation, we could be doing the API call using parameters *slug* or *symbol*, but symbol would return every cryptocurrency associated to the value passed (for example, by searching for *BTC* we would find other cryptocurrencies beyond the usual bitcoin) 

In [8]:
# defining the full query URL
metadata_url= base_url+'/v2/cryptocurrency/info'

# defining the appropriate parameters (I want to see basic information involving some of the main recent cryptocurrencies)
parameters= {
    'symbol': 'BTC,ETH,USDT,USDC,BNB,XRP,ADA,BUSD,SOL,DOT'
}
# Here we are using the symbol parameter in order to include 'Binance USD (BUSD)', 'Binance Coin (BNB)' and 'USD Coin (USDC)' in the search since their names have spaces and we could not specify them on slug argument. 

# But by doing it is important to emphasize a thing: using symbol parameter, the API will return to us any reference it has for such symbol stored in the endpoint we are calling (and it might not necessarily be a crypto). So we'll have to filter it later on

# making the call attempt
response= r.get(
    url= metadata_url
    ,params= parameters
    ,headers= headers
)

print(response.status_code)
print(response.reason)

# printing the entire response to take a look at the error message if there's a HTTP expection of type 4xx (i.e., user inconsistency)
if str(response.status_code)[0] == '4':
    print(response.json()) 

200
OK


In [10]:
# special treatment to handle all returned data and compile it on a single dataframe
crypto_symbols= ['BTC','ETH','USDT','USDC','BNB','XRP','ADA','BUSD','SOL','DOT']
df_data= []

for crypto in crypto_symbols:
    aux_df= pd.json_normalize(response.json()['data'][crypto])
    df_data.append(aux_df)

df= pd.concat(df_data, ignore_index= True)
df.head()

C:\Users\LUCAS\AppData\Local\Temp\ipykernel_18952\3920442243.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df= pd.concat(df_data, ignore_index= True)


,id,name,symbol,category,description,slug,logo,subreddit,notice,tags,...,urls.explorer,urls.reddit,urls.technical_doc,urls.source_code,urls.announcement,platform.id,platform.name,platform.slug,platform.symbol,platform.token_address
0,1,Bitcoin,BTC,coin,Bitcoin (BTC) is a cryptocurrency launched in ...,bitcoin,https://s2.coinmarketcap.com/static/img/coins/...,bitcoin,,"[mineable, pow, sha-256, store-of-value, state...",...,"[https://blockchain.info/, https://live.blockc...",[https://reddit.com/r/bitcoin],[https://bitcoin.org/bitcoin.pdf],[https://github.com/bitcoin/bitcoin],[],NaN,NaN,NaN,NaN,NaN
1,34316,HarryPotterTrumpSonic100Inu,BTC,token,HarryPotterTrumpSonic100Inu (BTC) is a cryptoc...,harrypottertrumpsonic100inu,https://s2.coinmarketcap.com/static/img/coins/...,,,[memes],...,[https://etherscan.io/token/0x7099ab9e42fa7327...,[],[],[],[],1027,Ethereum,ethereum,ETH,0x7099aB9E42Fa7327a6b15E0a0c120c3e50d11BeC
2,31652,batcat,BTC,token,batcat (BTC) is a cryptocurrency launched in 2...,batcat,https://s2.coinmarketcap.com/static/img/coins/...,,,"[memes, solana-ecosystem]",...,[https://solscan.io/token/EtBc6gkCvsB9c6f5wSbw...,[],[],[],[],5426,Solana,solana,SOL,EtBc6gkCvsB9c6f5wSbwG8wPjRqXMB5euptK6bqG1R4X
3,30938,Satoshi Pumpomoto,BTC,token,Satoshi Pumpomoto (BTC) is a cryptocurrency la...,satoshi-pumpomoto,https://s2.coinmarketcap.com/static/img/coins/...,,,None,...,[https://solscan.io/token/6AGNtEgBE2jph1bWFdya...,[],[],[],[],5426,Solana,solana,SOL,6AGNtEgBE2jph1bWFdyaqsXJ762emaP9RE17kKxEsfiV
4,31469,Boost Trump Campaign,BTC,token,Boost Trump Campaign (BTC) is a cryptocurrency...,boost-trump-campaign,https://s2.coinmarketcap.com/static/img/coins/...,,,None,...,[https://etherscan.io/token/0x300e0d87f8c95d90...,[],[],[],[],1027,Ethereum,ethereum,ETH,0x300e0d87f8c95d90cfe4b809baa3a6c90e83b850


In [11]:
df['category'].unique() # checking different categories of the extracted data

array(['coin', 'token'], dtype=object)

- As we previously mentioned, by filtering the query with the *symbol* argument we allow it to return to us different results that are not exclusively related to cryptos. We have te filter that

In [12]:
df= df[df['category'] == 'coin'] # applying filter on DataFrame

In [13]:
df['category'].unique() # checking the result

array(['coin'], dtype=object)

In [14]:
df.isna().sum() # verifying the quantity of null values inside the dataframe

id                                  0
name                                0
symbol                              0
category                            0
description                         0
slug                                0
logo                                0
subreddit                           0
notice                              0
tags                                0
tag-names                           0
tag-groups                          0
platform                            8
date_added                          0
twitter_username                    0
is_hidden                           0
date_launched                       5
contract_address                    0
self_reported_circulating_supply    8
self_reported_tags                  6
self_reported_market_cap            8
infinite_supply                     0
urls.website                        0
urls.twitter                        0
urls.message_board                  0
urls.chat                           0
urls.faceboo

In [16]:
# saving a final csv file
df.to_csv('API-data/cryptocurrency_metada.csv', sep= ';', index= False)

### 2) Cryptocurrency CoinMarketCap Map
- Updated only as needed every 30s 
  (se mudar/entrar alguma coisa, atualiza; senão não)

In [73]:
map_url= base_url+'/v1/cryptocurrency/map'

# 'symbol': 'BTC,ETH,USDT,USDC,BNB,XRP,ADA,BUSD,SOL,DOT'
# they strongly don't recommend us do the queries by symbol (because they can be duplicate for different cryptocurrencias and may also change over time for the same crypto)

# so I take the first 1000 crypto's data ordered according api developers rank
parameters={
    'limit': 10,
    'sort': 'cmc_rank'    
}

response= r.get(
    url= map_url,
    params= parameters,
    headers= headers
)

print(response.status_code)
print(response.reason)

if str(response.status_code)[0] != '2':
    print(response.text)
    # e-mail informando

response.json()['status'] # logs


200
OK


{'timestamp': '2025-01-02T21:41:40.771Z',
 'error_code': 0,
 'error_message': None,
 'elapsed': 61,
 'credit_count': 1,
 'notice': None}

In [77]:
pd.json_normalize(response.json()['data']).to_csv('API-data/top10_map_cryptos.csv', sep= ';', index= False)

#### 3) Cryptocurrency listing
- Update frequency: every 60s

In [ ]:
listing_url= base_url+'/v1/cryptocurrency/listings/latest'
aux= [] # lista auxiliar para armazenar os dataframes de iteração
#df= pd.DataFrame() # dataframe vazio

i=1
while True:
    
    print(f'Efetuando chamada dos índices {i} até {i+1000}')
    response= r.get(
        url= listing_url,
        headers= headers,
        params= { 'start': i, 'limit': 1000, 'aux': 'cmc_rank'}
    )
    
    print(f'Status Code: {response.status_code}')
    print(f'Log: {response.reason}')
    print(f'Créditos consumidos: {response.json()['status']['credit_count']}')

    # se não tivermos mais nenhum retorno, encerra o loop
    if len(response.json()['data']) == 0:
        break
    
    aux.append(pd.json_normalize(response.json()['data']))
    i+= 1000 # 1000 para contemplar as cryptos já retornadas
    print('-'*30, end='\n\n')

Efetuando chamada dos índices 1 até 1001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 1001 até 2001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 2001 até 3001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 3001 até 4001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 4001 até 5001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 5001 até 6001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 6001 até 7001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando chamada dos índices 7001 até 8001
Status Code: 200
Log: OK
Créditos consumidos: 5
------------------------------

Efetuando c

In [63]:
df= pd.concat(aux)
df.head()

C:\Users\LUCAS\AppData\Local\Temp\ipykernel_1220\901218407.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df= pd.concat(aux)


,id,name,symbol,slug,infinite_supply,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,...,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated
0,1,Bitcoin,BTC,bitcoin,False,1,NaN,NaN,NaN,2025-01-02T21:34:00.000Z,...,2.484410,1.423190,1.074754,40.200864,55.496827,1.920714e+12,56.4113,2.036625e+12,NaN,2025-01-02T21:34:00.000Z
1,1027,Ethereum,ETH,ethereum,True,2,NaN,NaN,NaN,2025-01-02T21:35:00.000Z,...,2.611236,3.670121,-4.883505,39.546444,42.096703,4.150576e+11,12.1936,4.150576e+11,NaN,2025-01-02T21:35:00.000Z
2,825,Tether USDt,USDT,tether,True,3,NaN,NaN,NaN,2025-01-02T21:35:00.000Z,...,0.093204,-0.011896,-0.188113,-0.072570,-0.132810,1.372737e+11,4.0323,1.412766e+11,NaN,2025-01-02T21:35:00.000Z
3,52,XRP,XRP,xrp,False,4,NaN,NaN,NaN,2025-01-02T21:35:00.000Z,...,2.939480,10.462372,-10.712960,371.537428,344.270390,1.363700e+11,4.0057,2.375360e+11,NaN,2025-01-02T21:35:00.000Z
4,1839,BNB,BNB,bnb,False,5,NaN,NaN,NaN,2025-01-02T21:35:00.000Z,...,-0.248000,2.245785,-0.472861,26.048530,26.607997,1.014323e+11,2.9795,1.014323e+11,NaN,2025-01-02T21:35:00.000Z


In [78]:
df[['id', 'name', 'symbol', 'cmc_rank']].sort_values(by='cmc_rank', ascending= True).head(11).to_csv('API-data/top10_cryptos.csv', sep=';', index=False)

In [ ]:
# defining the full query URL
listing_url= base_url+'/v1/cryptocurrency/listings/latest'

# defining the appropriate parameters (I want to see basic information involving some of the main recent cryptocurrencies)
#'symbol': 'BTC,ETH,USDT,USDC,BNB,XRP,ADA,BUSD,SOL,DOT',
parameters= {
    'limit': 1000,
    'cryptocurrency_type': 'coins'
}
# (bringing top 1000 cryptocurrencies ranked by marketing cap)

# making the call attempt
response= r.get(
    url= listing_url
    ,params= parameters
    ,headers= headers
)

print(response.status_code)
print(response.reason)

# printing the entire response to take a look at the error message if there's a HTTP expection of type 4xx (i.e., user inconsistency)
if str(response.status_code)[0] == '4':
    print(response.json()) 

200
OK


In [ ]:
quotes= pd.json_normalize(response.json()['data'])
quotes.head(n=3)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated
0,1,Bitcoin,BTC,bitcoin,11853,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.980477e+07,1.980477e+07,...,3.047018,1.517611,2.135381,41.916984,56.120474,1.928782e+12,56.5360,2.045185e+12,NaN,2025-01-02T20:03:00.000Z
1,1027,Ethereum,ETH,ethereum,9773,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.204752e+08,1.204752e+08,...,3.328444,3.759765,-3.219741,41.400741,42.604119,4.172501e+11,12.2222,4.172501e+11,NaN,2025-01-02T20:04:00.000Z
2,52,XRP,XRP,xrp,1491,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",1.000000e+11,5.741023e+10,9.998674e+10,...,3.656093,10.410152,-6.776671,377.111049,346.142798,1.372434e+11,4.0222,2.390574e+11,NaN,2025-01-02T20:04:00.000Z


In [30]:
quotes.dtypes

id                                      int64
name                                   object
symbol                                 object
slug                                   object
num_market_pairs                        int64
date_added                             object
tags                                   object
max_supply                            float64
circulating_supply                    float64
total_supply                          float64
infinite_supply                          bool
platform                               object
cmc_rank                                int64
self_reported_circulating_supply      float64
self_reported_market_cap              float64
tvl_ratio                             float64
last_updated                           object
quote.USD.price                       float64
quote.USD.volume_24h                  float64
quote.USD.volume_change_24h           float64
quote.USD.percent_change_1h           float64
quote.USD.percent_change_24h      

In [27]:
# filtering for the cryptos we have interest
quotes= quotes[quotes['symbol'].isin(['BTC','ETH','USDT','USDC','BNB','XRP','ADA','BUSD','SOL','DOT'])]
quotes

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated
0,1,Bitcoin,BTC,bitcoin,11853,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.980477e+07,1.980477e+07,...,3.047018,1.517611,2.135381,41.916984,56.120474,1.928782e+12,56.5360,2.045185e+12,NaN,2025-01-02T20:03:00.000Z
1,1027,Ethereum,ETH,ethereum,9773,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.204752e+08,1.204752e+08,...,3.328444,3.759765,-3.219741,41.400741,42.604119,4.172501e+11,12.2222,4.172501e+11,NaN,2025-01-02T20:04:00.000Z
2,52,XRP,XRP,xrp,1491,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",1.000000e+11,5.741023e+10,9.998674e+10,...,3.656093,10.410152,-6.776671,377.111049,346.142798,1.372434e+11,4.0222,2.390574e+11,NaN,2025-01-02T20:04:00.000Z
3,1839,BNB,BNB,bnb,2323,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.440069e+08,1.440069e+08,...,-0.265576,1.945406,2.091914,26.566837,26.485675,1.015120e+11,2.9750,1.015120e+11,NaN,2025-01-02T20:04:00.000Z
4,5426,Solana,SOL,solana,817,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.828132e+08,5.913368e+08,...,7.115762,9.396252,-9.400342,28.351905,43.823042,9.989180e+10,2.9261,1.223448e+11,NaN,2025-01-02T20:04:00.000Z
6,2010,Cardano,ADA,cardano,1425,2017-10-01T00:00:00.000Z,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.513978e+10,4.499513e+10,...,4.378805,11.069639,-19.186979,187.463577,171.874578,3.370696e+10,0.9874,4.316513e+10,NaN,2025-01-02T20:04:00.000Z
13,6636,Polkadot,DOT,polkadot-new,855,2020-08-19T00:00:00.000Z,"[substrate, polkadot, polkadot-ecosystem, thre...",NaN,1.534942e+09,1.534942e+09,...,2.076861,2.800571,-25.074203,89.573914,71.331861,1.100204e+10,0.3223,1.100204e+10,NaN,2025-01-02T20:04:00.000Z


In [29]:
# defining a new dataframe with only the columns we want to perform studies
df= quotes[
    [
        'id', # crypto Id
        'name', # crypto name
        'symbol', # crypto symbol
        'date_added', # date in which it was added to coinmarket cap database
        'last_updated', # last updated date in coinmarket cap database 
        'circulating_supply', # approximate number of coins in circulation 
        'total_supply', # estimative of total number of coins that exist at the moment
        'quote.USD.price', # estimative of the coin price in dollars
        'quote.USD.volume_24h', # total trading volume over last 24h
        'quote.USD.market_cap', # indicates the rank of the cryptocurrency based on its market capitalization
        'quote.USD.percent_change_7d' # percentage change in the trading price over the last 7 day 
    ]
]

,id,name,symbol,date_added,last_updated,circulating_supply,total_supply,quote.USD.price,quote.USD.volume_24h,quote.USD.market_cap,quote.USD.percent_change_7d
0,1,Bitcoin,BTC,2010-07-13T00:00:00.000Z,2025-01-02T20:03:00.000Z,1.980477e+07,1.980477e+07,97389.753029,4.582249e+10,1.928782e+12,1.517611
1,1027,Ethereum,ETH,2015-08-07T00:00:00.000Z,2025-01-02T20:04:00.000Z,1.204752e+08,1.204752e+08,3463.368692,2.242816e+10,4.172501e+11,3.759765
2,52,XRP,XRP,2013-08-04T00:00:00.000Z,2025-01-02T20:04:00.000Z,5.741023e+10,9.998674e+10,2.390574,9.143487e+09,1.372434e+11,10.410152
3,1839,BNB,BNB,2017-07-25T00:00:00.000Z,2025-01-02T20:04:00.000Z,1.440069e+08,1.440069e+08,704.910879,1.884816e+09,1.015120e+11,1.945406
4,5426,Solana,SOL,2020-04-10T00:00:00.000Z,2025-01-02T20:04:00.000Z,4.828132e+08,5.913368e+08,206.895337,4.289774e+09,9.989180e+10,9.396252
6,2010,Cardano,ADA,2017-10-01T00:00:00.000Z,2025-01-02T20:04:00.000Z,3.513978e+10,4.499513e+10,0.959225,1.367080e+09,3.370696e+10,11.069639
13,6636,Polkadot,DOT,2020-08-19T00:00:00.000Z,2025-01-02T20:04:00.000Z,1.534942e+09,1.534942e+09,7.167723,3.152830e+08,1.100204e+10,2.800571


### 3.1) Data Cleaning and Preparing